# 준비
라이브러리, 파라미터 세팅

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import re

In [2]:
DATA_IN_PATH = './data_in/'
DATA_OUT_PATH = './data_out/99_SubmissionFiles_for_Ensemble_ML/'

TEST_SIZE = 0.2
RANDOM_SEED = 42

# 데이터 불러오기

In [3]:
# 테스트(캐글) 데이터 로드
df_ensemble = pd.read_csv(DATA_IN_PATH + 'en_data.csv')
df_ensemble.drop('i_dialog', axis=1, inplace=True)
df_ensemble.drop('i_utterance', axis=1, inplace=True)
df_ensemble.drop('speaker', axis=1, inplace=True)
df_ensemble.columns = ['Id','utterance']  # 컬럼명 변경
df_ensemble.head()

,Id,utterance
0,0,"Alright, whadyou do with him?"
1,1,Oh! You're awake!
2,2,Then you gotta come clean with Ma! This is not...
3,3,"Yeah, but this is"
4,4,I don't wanna hear it! Now go to my room!


In [5]:
# 앙상블 대상 파일 확인
fnames = glob.glob(DATA_OUT_PATH+'*.csv')
fnames.sort()

for i in fnames:
    print(i.split('/')[-1])

FRIENDS_LightGBM_045006.csv
FRIENDS_LogisticRegression_044759.csv
FRIENDS_RandomForest_044266.csv
FRIENDS_XGBoost_044019.csv


In [5]:
len(fnames)

4

In [6]:
for i in range(len(fnames)):
    tmp = pd.read_csv(fnames[i])
    tmp.drop('Id', axis=1, inplace=True)
    df_ensemble = pd.concat([df_ensemble, tmp], axis=1)

In [7]:
df_ensemble

,Id,utterance,Predicted,Predicted,Predicted,Predicted
0,0,"Alright, whadyou do with him?",neutral,neutral,neutral,neutral
1,1,Oh! You're awake!,neutral,neutral,neutral,neutral
2,2,Then you gotta come clean with Ma! This is not...,neutral,neutral,neutral,neutral
3,3,"Yeah, but this is",neutral,neutral,neutral,neutral
4,4,I don't wanna hear it! Now go to my room!,neutral,neutral,non-neutral,neutral
...,...,...,...,...,...,...
1618,1618,Nooo.,neutral,non-neutral,non-neutral,neutral
1619,1619,"Hi, Kate!",neutral,neutral,neutral,neutral
1620,1620,"Hi, Lauren.",neutral,neutral,neutral,neutral
1621,1621,"Hi, Lauren.",neutral,neutral,neutral,neutral


# 모델 앙상블(Majority Voting) 방식

In [18]:
ensemble_result = []

for i in range(len(df_ensemble)):  # Id 별
    cnt_neutral = 0
    cnt_non_neutral = 0
    cnt_joy = 0
    cnt_sadness = 0
    cnt_fear = 0
    cnt_anger = 0
    cnt_surprise = 0
    cnt_disgust = 0
    
    for j in range(2,len(fnames)+2):  # 모델 별 (for j in range(2,len(fnames)+1): ?)
        if df_ensemble.iloc[i,j] == 'neutral':
            cnt_neutral += 1
        if df_ensemble.iloc[i,j] == 'non-neutral':
            cnt_non_neutral += 1
        if df_ensemble.iloc[i,j] == 'joy':
            cnt_joy += 1
        if df_ensemble.iloc[i,j] == 'sadness':
            cnt_sadness += 1
        if df_ensemble.iloc[i,j] == 'fear':
            cnt_fear += 1
        if df_ensemble.iloc[i,j] == 'anger':
            cnt_anger += 1
        if df_ensemble.iloc[i,j] == 'surprise':
            cnt_surprise += 1
        if df_ensemble.iloc[i,j] == 'disgust':
            cnt_disgust += 1            
    
    print(i, 'neutral:', cnt_neutral \
          ,  'non-neutral:', cnt_non_neutral \
          ,  'joy:', cnt_joy \
          ,  'sadness:', cnt_sadness \
          ,  'fear:', cnt_fear \
          ,  'anger:', cnt_anger \
          ,  'surprise:', cnt_surprise \
          ,  'disgust:', cnt_disgust \
         )
    
#     # 0.44143 => 학습(EmoLex 감성사전 추가 가정) 데이터에서 빈도가 가장 높은 것에서 낮은 순으로 정렬
#     if cnt_neutral >= int(len(fnames)/2):  ensemble_result.append('neutral')
#     elif cnt_non_neutral >= int(len(fnames)/2):  ensemble_result.append('non-neutral')
#     elif cnt_joy >= int(len(fnames)/2):  ensemble_result.append('joy')
#     elif cnt_surprise >= int(len(fnames)/2):  ensemble_result.append('surprise')
#     elif cnt_fear >= int(len(fnames)/2):  ensemble_result.append('fear')
#     elif cnt_sadness >= int(len(fnames)/2):  ensemble_result.append('sadness')
#     elif cnt_disgust >= int(len(fnames)/2):  ensemble_result.append('disgust')
#     elif cnt_anger >= int(len(fnames)/2):  ensemble_result.append('anger')
#     else:  ensemble_result.append('non-neutral')  # 특정 감정으로 단정할 수 없으며 중립으로 보기 애매하다고 세팅

#     # 0.44143 => 학습(기본으로 주어진) 데이터에서 빈도가 가장 높은 것에서 낮은 순으로 정렬
#     if cnt_neutral >= 2:  ensemble_result.append('neutral')
#     elif cnt_non_neutral >= 2:  ensemble_result.append('non-neutral')
#     elif cnt_joy >= 2:  ensemble_result.append('joy')
#     elif cnt_surprise >= 2:  ensemble_result.append('surprise')
#     elif cnt_anger >= 2:  ensemble_result.append('anger')
#     elif cnt_sadness >= 2:  ensemble_result.append('sadness')
#     elif cnt_disgust >= 2:  ensemble_result.append('disgust')
#     elif cnt_fear >= 2:  ensemble_result.append('fear')
#     else:  ensemble_result.append('non-neutral')  # 특정 감정으로 단정할 수 없으며 중립으로 보기 애매하다고 세팅

    # 0.44389 => 학습(기본으로 주어진) 데이터에서 빈도가 가장 높은 것에서 낮은 순으로 정렬
    if cnt_neutral >= 3:  ensemble_result.append('neutral')
    elif cnt_non_neutral >= 3:  ensemble_result.append('non-neutral')
    elif cnt_joy >= 3:  ensemble_result.append('joy')
    elif cnt_surprise >= 3:  ensemble_result.append('surprise')
    elif cnt_anger >= 3:  ensemble_result.append('anger')
    elif cnt_sadness >= 3:  ensemble_result.append('sadness')
    elif cnt_disgust >= 3:  ensemble_result.append('disgust')
    elif cnt_fear >= 3:  ensemble_result.append('fear')
    else:  ensemble_result.append('non-neutral')  # 특정 감정으로 단정할 수 없으며 중립으로 보기 애매하다고 세팅

df_ensemble["Ensemble_Result"] = ensemble_result

0 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
2 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
3 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
4 neutral: 3 non-neutral: 1 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
5 neutral: 3 non-neutral: 0 joy: 0 sadness: 1 fear: 0 anger: 0 surprise: 0 disgust: 0
6 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
7 neutral: 3 non-neutral: 1 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
8 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
9 neutral: 1 non-neutral: 3 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
10 neutral: 1 non-neutral: 3 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
11 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 

116 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
117 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
118 neutral: 2 non-neutral: 0 joy: 2 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
119 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
120 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
121 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
122 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
123 neutral: 3 non-neutral: 1 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
124 neutral: 2 non-neutral: 0 joy: 2 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
125 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
126 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
127 neutral: 4 non-neutral: 0 jo

219 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
220 neutral: 1 non-neutral: 3 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
221 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
222 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
223 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
224 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
225 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
226 neutral: 3 non-neutral: 0 joy: 0 sadness: 1 fear: 0 anger: 0 surprise: 0 disgust: 0
227 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
228 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
229 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
230 neutral: 4 non-neutral: 0 jo

326 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
327 neutral: 1 non-neutral: 3 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
328 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
329 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
330 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
331 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
332 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
333 neutral: 1 non-neutral: 2 joy: 1 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
334 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
335 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
336 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
337 neutral: 4 non-neutral: 0 jo

432 neutral: 2 non-neutral: 0 joy: 2 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
433 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
434 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
435 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
436 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
437 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
438 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
439 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
440 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
441 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
442 neutral: 2 non-neutral: 0 joy: 2 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
443 neutral: 0 non-neutral: 0 jo

...

1180 neutral: 2 non-neutral: 1 joy: 1 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1181 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1182 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1183 neutral: 3 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 1 disgust: 0
1184 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1185 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1186 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1187 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1188 neutral: 0 non-neutral: 2 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 2 disgust: 0
1189 neutral: 0 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 4 disgust: 0
1190 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1191 neutral: 4 non-n

1287 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1288 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1289 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1290 neutral: 1 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 3 disgust: 0
1291 neutral: 2 non-neutral: 1 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 1 disgust: 0
1292 neutral: 0 non-neutral: 2 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 2 disgust: 0
1293 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1294 neutral: 1 non-neutral: 3 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1295 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1296 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1297 neutral: 3 non-neutral: 0 joy: 0 sadness: 1 fear: 0 anger: 0 surprise: 0 disgust: 0
1298 neutral: 4 non-n

1394 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1395 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1396 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1397 neutral: 3 non-neutral: 0 joy: 0 sadness: 1 fear: 0 anger: 0 surprise: 0 disgust: 0
1398 neutral: 0 non-neutral: 4 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1399 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1400 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1401 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1402 neutral: 1 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 3 disgust: 0
1403 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1404 neutral: 0 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 4 disgust: 0
1405 neutral: 1 non-n

1501 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1502 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1503 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1504 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1505 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1506 neutral: 1 non-neutral: 3 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1507 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1508 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1509 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1510 neutral: 1 non-neutral: 2 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 1 disgust: 0
1511 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1512 neutral: 4 non-n

1608 neutral: 3 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 1 disgust: 0
1609 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1610 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1611 neutral: 1 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 3 disgust: 0
1612 neutral: 2 non-neutral: 2 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1613 neutral: 0 non-neutral: 4 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1614 neutral: 3 non-neutral: 1 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1615 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1616 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1617 neutral: 3 non-neutral: 1 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1618 neutral: 2 non-neutral: 2 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1619 neutral: 4 non-n

In [19]:
df_ensemble.head(20)

,Id,utterance,Predicted,Predicted,Predicted,Predicted,Ensemble_Result
0,0,"Alright, whadyou do with him?",neutral,neutral,neutral,neutral,neutral
1,1,Oh! You're awake!,neutral,neutral,neutral,neutral,neutral
2,2,Then you gotta come clean with Ma! This is not...,neutral,neutral,neutral,neutral,neutral
3,3,"Yeah, but this is",neutral,neutral,neutral,neutral,neutral
4,4,I don't wanna hear it! Now go to my room!,neutral,neutral,non-neutral,neutral,neutral
5,5,I don?t want him to tell this story for years.,sadness,neutral,neutral,neutral,neutral
6,6,"Oh, but he will. He still tells the story how ...",neutral,neutral,neutral,neutral,neutral
7,7,I wasn?t escaping.,non-neutral,neutral,neutral,neutral,neutral
8,8,Then how did you get caught in the barbed wire?,neutral,neutral,neutral,neutral,neutral
9,9,I was trying to help out a squirrel.,non-neutral,non-neutral,non-neutral,neutral,non-neutral


In [20]:
df_ensemble.tail(20)

,Id,utterance,Predicted,Predicted,Predicted,Predicted,Ensemble_Result
1603,1603,Yeah.,neutral,neutral,neutral,neutral,neutral
1604,1604,Hey!,neutral,neutral,neutral,neutral,neutral
1605,1605,Hey.,neutral,neutral,neutral,neutral,neutral
1606,1606,What are you wrapping?,neutral,neutral,neutral,neutral,neutral
1607,1607,"Oh-ho, look what I got Julio.",joy,joy,non-neutral,joy,joy
1608,1608,"Yeah, it's a vase.",neutral,surprise,neutral,neutral,neutral
1609,1609,"Yeah, just like the one in the poem.",neutral,neutral,neutral,neutral,neutral
1610,1610,Well not exactly like the one in the poem.,neutral,neutral,neutral,neutral,neutral
1611,1611,What are you talking about?,surprise,neutral,surprise,surprise,surprise
1612,1612,It was a big deal.,non-neutral,neutral,non-neutral,neutral,non-neutral


In [21]:
output = df_ensemble[['Id','Ensemble_Result']].copy()
output = output.rename({'Ensemble_Result':'Predicted'}, axis='columns')
output.head()

,Id,Predicted
0,0,neutral
1,1,neutral
2,2,neutral
3,3,neutral
4,4,neutral


In [22]:
# 해당 경로가 없으면 생성
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

# csv파일 생성
output.to_csv("./data_out/FRIENDS_Ensemble_ML.csv", index = False)  # 앙상블 결과, 캐글 제출 용도

### 캐글 제출 결과
**[2020.12.18]**<br>
FRIENDS_LightGBM_045006.csv<br>
FRIENDS_LogisticRegression_044759.csv<br>
FRIENDS_RandomForest_044266.csv<br>
FRIENDS_XGBoost_044019.csv<br>
조합(4개) => 0.44389<br>

개별 모델 결과보다 성능 향상이 잘 되지 않음